## Advent of Code - Day 1

- DuckDB SQL version
- Python version (initial and enhanced)
- Polars version

*Note: Space separators led to a few issues with reading the data file.* 

In [1]:
import duckdb
import polars as pl

In [2]:
DAY1_DATA = "../data/day1_lists.txt"

In [3]:
# Quick check on the data

!head -5 ../data/day1_lists.txt

77710   11556
22632   23674
82229   77288
35788   30924
84000   63702


## DuckDB SQL method

I thought I'd experiment with a SQL-based approach as a bit of a learning / comparison opportunity.

In [4]:
sql_query = f"""
WITH data AS (
  SELECT 
    CAST(column0 AS INTEGER) AS L1,
    CAST(column1 AS INTEGER) AS L2
  FROM read_csv_auto('{DAY1_DATA}', header=false, delim=' ')
)
, sorted_L1 AS (
  SELECT L1, ROW_NUMBER() OVER (ORDER BY L1) AS idx FROM data
)
, sorted_L2 AS (
  SELECT L2, ROW_NUMBER() OVER (ORDER BY L2) AS idx FROM data
)
, paired_distances AS (
  SELECT 
    sL1.L1,
    sL2.L2,
    ABS(sL1.L1 - sL2.L2) AS distance
  FROM sorted_L1 sL1
  JOIN sorted_L2 sL2 ON sL1.idx = sL2.idx
)
, total_distance AS (
  SELECT SUM(distance) AS total_distance FROM paired_distances
)
SELECT total_distance FROM total_distance;
"""

In [5]:
conn = duckdb.connect()

# SQL script to run all steps, creating temp tables for each stage
sql_script = f"""
-- Create temporary table with selected columns
CREATE TEMPORARY TABLE data AS
SELECT 
  CAST(column0 AS INTEGER) AS L1,
  CAST(column3 AS INTEGER) AS L2
FROM read_csv_auto('{DAY1_DATA}', header=false, delim=' ');

-- Create temporary table with sorted L1
CREATE TEMPORARY TABLE sorted_L1 AS
SELECT L1, ROW_NUMBER() OVER (ORDER BY L1) AS idx FROM data;

-- Create temporary table with sorted L2
CREATE TEMPORARY TABLE sorted_L2 AS
SELECT L2, ROW_NUMBER() OVER (ORDER BY L2) AS idx FROM data;

-- Create temporary table with paired distances
CREATE TEMPORARY TABLE paired_distances AS
SELECT sL1.L1, sL2.L2, ABS(sL1.L1 - sL2.L2) AS distance
FROM sorted_L1 sL1
JOIN sorted_L2 sL2 USING (idx);

-- Create temporary table with total distance
CREATE TEMPORARY TABLE total_distance AS
SELECT SUM(distance) AS total_distance FROM paired_distances;
"""

# Execute the full pipeline SQL script
conn.execute(sql_script)

# Fetch and print the final result
result = conn.execute("SELECT * FROM total_distance").fetchdf()
print(f"Total Distance: {result.squeeze()}")

# Optional: Fetch and inspect intermediate tables if needed
# print(conn.execute("SELECT * FROM data LIMIT 5").fetchdf())
# print(conn.execute("SELECT * FROM sorted_L1 LIMIT 5").fetchdf())
# print(conn.execute("SELECT * FROM paired_distances LIMIT 5").fetchdf())

Total Distance: 1197984.0


## Pure Python method

In [6]:
# Load the lists to a list

L1 = []
L2 = []
with open(DAY1_DATA, "r") as f:
    for line in f:
        line = line.strip()
        if line:  # Skip empty lines
            parts = line.split()  # Splits on whitespace (spaces or tabs)
            if len(parts) == 2:
                L1.append(int(parts[0]))
                L2.append(int(parts[1]))
            else:
                print(f"Skipping invalid line: {line}")

# Now l1 and l2 are loaded
# For verification, print lengths and first few elements
# print(f"Length of L1: {len(L1)}")
# print(f"Length of L2: {len(L2)}")
# print(f"First 5 of L1: {L1[:5]}")
# print(f"First 5 of L2: {L2[:5]}")

In [7]:
L1 = sorted(L1)
L2 = sorted(L2)
diffs = [abs(l1 - l2) for l1, l2 in zip(L1, L2)]
total_distance = sum(diffs)
print(f"Total distance: {total_distance}")

Total distance: 1197984


Improved version of the initial method

In [8]:
# Read and parse the file in one step using list comprehension
with open(DAY1_DATA, "r") as f:
    data = [line.split() for line in f if line.strip() and len(line.split()) == 2]

# Separate the values into two lists and convert to int
L1, L2 = zip(*[(int(a), int(b)) for a, b in data])

# Convert tuples back to lists (optional, if you need to modify later)
L1 = list(L1)
L2 = list(L2)

# Sort the lists
L1.sort()
L2.sort()

# Calculate sum of absolute differences
total_distance = sum(abs(a - b) for a, b in zip(L1, L2))

# Debugging output
# print(f"Length of L1: {len(L1)}")
# print(f"Length of L2: {len(L2)}")
# print(f"First 5 of L1: {L1[:5]}")
# print(f"First 5 of L2: {L2[:5]}")

print(f"Total distance: {total_distance}")

Total distance: 1197984


## Polars version

In [9]:
# Read the space-separated file with no header
df = pl.read_csv(DAY1_DATA, separator=" ", has_header=False)

# Select relevant columns 1 and 4, rename and cast to int
df_selected = df.select(
    [
        pl.col("column_1").cast(pl.Int64).alias("L1"),
        pl.col("column_4").cast(pl.Int64).alias("L2"),
    ]
)

# Sort each list separately and add index
sorted_L1 = df_selected.select("L1").sort("L1").with_row_index("idx")
sorted_L2 = df_selected.select("L2").sort("L2").with_row_index("idx")

# Join sorted lists on index to pair elements
paired = sorted_L1.join(sorted_L2, on="idx")

# Calculate absolute differences and sum them
paired = paired.with_columns((pl.col("L1") - pl.col("L2")).abs().alias("distance"))
total_distance = paired["distance"].sum()

print("Total Distance:", total_distance)

Total Distance: 1197984
